# Kubernetes

# Kubernetes Version (Current)

In [ ]:
from kubernetesTool import executeKubectlCmd
from kubernetesTool import toCmd
from kubernetesTool import executeLocalCmd
from kubernetesTool import createConfig
from kubernetesTool import getPort
from kubernetesTool import executePeerCmd

In [277]:
#!pip install ipynb
from ipynb.fs.full.facturxWidget import *

Text(value='15', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Dropdown(description='FILTER', index=1, layout=Layout(width='500px'), options=(False, True), style=Description…

Dropdown(description='FILTER', index=2, layout=Layout(width='500px'), options=('/supplyChainTradeTransaction/a…

Output()

Output()

...

Text(value='15', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Output()

Text(value='14', description='invoice', layout=Layout(width='500px'), style=DescriptionStyle(description_width…

Output()

AttributeError: 'SimpleContainer' object has no attribute '_widget'

## check status peers

In [ ]:
from kubernetesTool import executeKubectlCmd
_res = executeKubectlCmd(['get', 'pods', '-o', 'json'], decode='json')

for item in _res['items']:
    labels=item['metadata']['labels']
    #status=item['status']['initContainerStatuses'][0]
    #print(labels,status['state'],status['restartCount'],status['name'],status['ready'])
    print(labels,item['status']['phase'])
    #print(labels)

In [ ]:
kubernetespeer('invoice-chaincode-external').__dict__

In [ ]:
_res['items'][3]['metadata']['labels']
_res['items'][0]['status']['phase']

In [ ]:
#30489
kubernetesTool.getPort(peername='invoice-chaincode-external', name = prefix + '-listen')

In [ ]:
kubernetespeer(name='invoice-net-chaincode-external').__dict__

In [ ]:
#_res['items'][5]['status']['containerStatuses']
_res['items'][5]

In [ ]:
for item in _res['items']:
    labels=item['metadata']['labels']
    if labels['app'] == 'invoice-net-chaincode-external':
        print(item)

## addPeer

In [ ]:
def addPeer(peername):  
    if peername in peers.keys():
        return peers[peername]
    # Create configuration for new peer
    executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray-ledger/containers/ && /home/jovyan/scray-ledger/containers/configure-deployment.sh -n ' + peername])
    # Start service
    executeKubectlCmd(['apply', '-f', '/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer-service.yaml'])
    # Create peer configuration
    res = createConfig(peername=peername,
                       peer_listen_port=getPort(peername=peername, name='peer-listen'),
                       peer_gossip_port=getPort(peername=peername, name='peer-gossip'))
    # Start new peer
    executeKubectlCmd(['apply', '-f','/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer.yaml'])
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer
    
# create, delete numbered peers
def createPeers(base_name,start=0, end=10):
    for x in range(start,end):
        peername = base_name + '-' + str(x)
        addPeer(peername)
         
def deletePeers(name):
    for x in range(0,10):
        peerneme = name + str(x)
        deletePeer(peerneme)    
  
def deleteAllPeers(config):
    for item in config['items']:
        try:
            if 'org_name' in item['data'].keys():
                deletePeer(item['data']['org_name'])
        except Exception as e:
            print(item['data'])
            

### config            
            
import subprocess
def deleteConfig(peername=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + peername])
        return str(output)
    except Exception as e:
        return str(e)  
        
    
def createConfig(peername='',peer_listen_port='',peer_gossip_port=''):
    try:  
        peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=' + peer_host_name,
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=data_share=hl-fabric-data-share-service:80',
                                          '--from-literal=CORE_PEER_ADDRESS=peer0.' + peer_host_name + ':' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=' + peer_host_name + ':' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)      
    
  
##### execute

def executePeerCmd(pod_name,cmd, cli='scray-peer-cli'):
    try:  
        #cmd1 = "source /root/.bashrc && "
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd]
        #cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c'] + cmd
        print(toCmd(cmd1))
        output  = subprocess.check_output(cmd1)
        #output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec', '--stdin' ,'--tty', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)

In [ ]:
import kubernetesTool

class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name
        self.pod_name = kubernetesTool.getPod(name)
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
        # ????????
        #print('kubernetespeer addBashEnvToPeer')
        #addBashEnvToPeer(self)
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)      
        
class ordererpeer(kubernetespeer):
    def __init__(self, name='',prefix='peer'):
        super().__init__(name,prefix)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

In [ ]:
kubernetespeer('buyer')

In [ ]:
def toCmd(strlist):
    return ''.join((str(e) + ' ') for e in strlist)[:-1]

#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME         
def joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip='10.15.136.41'):          
    cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
    print(cmd)
    return executePeerCmd(peer.pod_name, cmd)

def endorse_peer(peer=None, orderer_ip='',channel_name='',shared_fs_host='',ext_peer_ip='10.15.136.41'):
    print(peer.name)
    peername = peer.name
    peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
    cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,peer.name,shared_fs_host,ext_peer_ip,peer_host_name]) 
    #print(cmd)
    return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

def add_channel(name):
    cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050']
    cmd = toCmd(cmd1)
    return executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

## create peers

In [ ]:
orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')

orderer_ip=orderer.get_ip()
orderer_hostname=orderer.host
#orderer_port=orderer.orderer_intern
orderer_port= orderer.listen_port

channel_name='invoicing'
shared_fs_host='10.14.128.38:30080'

In [ ]:
def endorse_and_join(peer):    
    res1 = endorse_peer(peer=peer,orderer_ip=orderer_ip,channel_name=channel_name,shared_fs_host='10.14.128.38:30080')   
    res2  = joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host)
    return res1,res2

In [ ]:
peers = {}

In [ ]:
addPeer('aubonmoulin')
addPeer('lyon-impots')
addPeer('austriafactor')
addPeer('megustaolive') 

In [ ]:
#createPeers('babby',start=0, end=2)
createPeers('babby',start=2, end=4)

In [ ]:
## endorse and join

import ipywidgets as widgets
out2 = widgets.Output()

with out2:
    for key, value in peers.items():
        endorse_and_join(value)

In [ ]:
out2

In [ ]:
peername='zalya-3'
peer = kubernetespeer(name=peername)
peers[peername] = peer

In [ ]:
'zalya-0'

In [ ]:
peers['zalya-1'].__dict__
#peers

In [ ]:
addPeer('zalya-3')

In [ ]:
orderer.__dict__

In [ ]:
kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/inform_existing_nodes.sh $ORDERER_IP $CHANNEL_NAME $PEER_NAME $SHARED_FS_HOST $EXT_PEER_IP $PEER_HOST_NAME

In [ ]:
EER_POD_NAME=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
 ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
 ORDERER_PORT=7050
 PEER_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
 kubectl exec --stdin --tty $PEER_POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $ORDERER_PORT $CHANNEL_NAME $SHARED_FS_HOST $EXT_PEER_IP

In [ ]:
endorse_and_join(peers['zalya-3'])

In [ ]:
installAndAproveChaincode(peers['zalya-1'],orderer,channel_name)


In [ ]:
#out2.outputs[1]['text'].splitlines()

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-5f97c57d44-j5mbs -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super40 kalya-0 10.14.128.38:30080 10.15.136.41 kalya-0.kubernetes.research.dev.seeburger.de

In [ ]:
name='peer-gossip'
callopt= 'jsonpath=\"{.spec.ports[?(@.name==\'' + name + '\')].nodePort}\"'
callopt

## install chaincode

In [ ]:
def installAndAproveChaincode(peer,orderer,channel_name,cc_host='10.14.128.38',pkgid='',cc_hostname='asset-transfer-basic.org1.example.com',cc_label='basic_1.0',shared_fs='kubernetes.research.dev.seeburger.de:30080', sequence=1):
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK
    #strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.orderer_intern), channel_name ]
    strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.listen_port), channel_name,pkgid,cc_hostname, cc_label,shared_fs, str(sequence) ]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

def commitChaincode(peer,channel_name,sequence=1,pkgid=''):
    strlist = ['/mnt/conf/peer/cc_commit.sh', channel_name, str(sequence),pkgid ]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

In [ ]:
for key, peer in peers.items():
    executePeerCmd(peer.pod_name,'apk add curl')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/install_and_approve_cc.sh', '-o', '/mnt/conf/install_and_approve_cc.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/install_and_approve_cc.sh')
    strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/containers/hl-fabric-node-configurator/conf/peer/cc_commit.sh', '-o', '/mnt/conf/peer/cc_commit.sh']
    cmd = toCmd(strlist)
    print(cmd)
    output = executePeerCmd(peer.pod_name,  cmd)
    executePeerCmd(peer.pod_name,'chmod 755 /mnt/conf/peer/cc_commit.sh')

In [ ]:
 peernames

In [ ]:
shared_fs='kubernetes.research.dev.seeburger.de:30080'
cc_hostname='invoice-net.org1.example.com'
cc_host='10.14.128.38'
cc_service_name='hl-fabric-cc-external-invoice-net'
cc_port = executeKubectlCmd(['get', 'service', cc_service_name, '-o', 'jsonpath="{.spec.ports[?(@.name==\'chaincode\')].nodePort}"'], decode='json')
cc_label='basic_1.0'

In [ ]:
cc_port

In [ ]:
import json
strlist = ['curl','-s','--user','scray:scray','http://' + shared_fs + '/cc_descriptions/' + cc_hostname + '_' + cc_label + '/description-hash.json']
cmd = toCmd(strlist)
output = executePeerCmd(peer.pod_name,  cmd)
pkgid = json.loads(output[:-1])['description-hash']

In [ ]:
output

In [ ]:
for key, peer in peers.items():
    installAndAproveChaincode(peer,orderer,channel_name,cc_host=cc_host,pkgid=pkgid,cc_hostname=cc_hostname,cc_label=cc_label,shared_fs=shared_fs, sequence=1)

In [ ]:
peernames = list(peers.keys())
peernames
peer = peers[peernames[0]]
#peer = list(peers.values())[0]
commitChaincode(peer,channel_name,sequence=1,pkgid=pkgid)

In [ ]:
#channel_name
#orderer.__dict__
#peers['ralya-0'].pod_name
peers['zalya-0'].__dict__ , channel_name

In [ ]:
installAndAproveChaincode(peers['zalya-1'],orderer,channel_name,sequence=2)


In [ ]:
#commitChaincode(peers['zalya-1'],channel_name,sequence=2)
chaincode_init(peers['zalya-1'],channel_name)

In [ ]:
peers['zalya-2'].pod_name

In [ ]:
def downloadToPeer(peer, orgs, execute_org='schalke3'):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name=getCrtName(org))))   

In [ ]:
peers

In [ ]:
peer = peers['zalya-0']
executePeerCmd(peer.pod_name,'apk add curl')

In [ ]:
peer.pod_name

In [ ]:
peer=kubernetespeer(name='oalya-0')
channel_name='super45'

In [ ]:
peer.name, channel_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/cc-basic-interaction.sh  super43

In [ ]:
#Call init method in chaincode
def chaincode_init(peer,channel_name):
    strlist = ['/mnt/conf/peer/cc-basic-interaction.sh', channel_name ]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

chaincode_init(peer,channel_name)

In [ ]:
externalbuilder/builds/basic_1.0-21ee1256844f847e196af6aec17c5cafa76d8bd652d6ef42c3058e52571030b4/release/chaincode/server/c

In [ ]:
peers['nalya-0'].pod_name

In [ ]:
CHANNEL_NAME='super44'
#ORDERER_NAME=orderer.example.com
IP_CC_SERVICE='10.14.128.38'         # Host where the chaincode is running
#PEER_POD=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
#ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')
#ORDERER_LISTEN_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
#ORDERER_HOST=orderer.example.com
EXT_PEER_IP="10.14.128.38"
PEER_POD="nalya-0-5494459f47-v7msj"

ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
ORDERER_PORT=30081
ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')

kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $ORDERER_IP $ORDERER_HOST $ORDERER_PORT $CHANNEL_NAME 

In [ ]:
#peers.keys()
#[elem for elem in peers.values()]
list(peers.values())[0].__dict__

# invoice usecase

In [ ]:
peernames = list(peers.keys())
peernames

## invoice functions

In [ ]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh  $CHANNEL_NAME $INVOICE_ID

def get_certificate_string(peername,user='User1'):
    return 'x509::CN=' + user + '@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

def chaincode_add_invoice(peer,channel_name,invoice_id,buyer):
    strlist = ['/mnt/conf/peer/add-invoice.sh', channel_name,invoice_id,get_certificate_string(buyer)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 


In [ ]:
import json

def getHostname(peer,user):
    splits = user.split('@')
    if len(splits) > 1:
        return splits[1]
    return executePeerCmd(peer.pod_name,'echo $HOSTNAME')[:-1]

def getExport_MSPCONFIGPATH(hostname,user='User1'):
    splits = user.split('@')
    if len(splits) > 1:
        user = splits[0]
    #user = 'User1'
    #user = 'hans'
    #path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/User1@' + hostname + '/msp/'
    path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + user + '@' + hostname + '/msp/'
    return 'export CORE_PEER_MSPCONFIGPATH=' + path
    
def getArgList(asset):
    if isinstance(asset, dict):
        asset = asset.values()
    return '[' + ''.join(('\"' + str(e) + '\",') for e in asset)[:-1] + ']' 

def invoke(peer,callopt,user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
            cmd0 = str('\'' + callopt + "\'")
            strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)  
            output = executePeerCmd(peer.pod_name, cmd)
            if output.endswith ('\n'):
                return output[:-1],_out
            return output,_out
        except Exception as e:
            print('Exception:', e)
            if output.endswith ('\n'):
                return output[:-1],_out
            return output,_out
        
def query(peer, callopt, user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
            #hostname = getHostname(peer)
            #path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + 'User1' + '@' + hostname + '/msp/'
            cmd0 = str('\'' + callopt + "\'")
            strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0]
            #strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0,'--cafile',path + 'cacerts/ca.zalya-0.kubernetes.research.dev.seeburger.de-cert.pem' , '--certfile',path + 'signcerts/User1@zalya-0.kubernetes.research.dev.seeburger.de-cert.pem' ,'--keyfile', path + 'keystore/priv_sk']
            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)
            output = executePeerCmd(peer.pod_name, cmd)
            #output = executePeerCmd(peer.pod_name, cmd)  
            if output.endswith ('\n'):
                output=output[:-1]
            return json.loads(output),_out
        except Exception as e:
            print('Exception:', e)
            return output,_out
    
def getAllKeys(peer):
    callopt= '{"function":"GetAllKeys","Args":[' + ']}'
    return query(peer,callopt)       

def getRoleTransactions(peer):
    callopt= '{"function":"GetRoleTransactions","Args":[' + ']}'
    return query(peer,callopt)   
    
def getEmptyInvoice(peer):
    callopt= '{"function":"GetEmptyInvoice","Args":[' + ']}'
    return query(peer,callopt)       
    
def createInvoice(peer,arglist='', user='User1'):
    callopt= '{"function":"CreateInvoice","Args":' + arglist + '}'
    return invoke(peer,callopt,user=user)    
    
def listInvoices(peer, user='User1'):
    callopt= '{"function":"ListInvoices","Args":[' + ']}'
    return query(peer,callopt,user=user)    
    
def listInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ListInvoice","Args":' + getArgList([id]) + '}'
    return query(peer,callopt,user=user)    
    
def receivedInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedInvoice","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)     
    
def receivedOrder(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedOrder","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)   

def receivedPayment(peer, id='asset1', payer='', user='User1'):
    callopt= '{"function":"ReceivedPayment","Args":' + getArgList([id,payer]) + '}'
    return invoke(peer,callopt,user=user)   

def taxReceived(peer, id='asset1', user='User1'):
    callopt= '{"function":"TaxReceived","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)    
    
    
def assetExists(peer, id='asset1'):
    #callopt= '{"function":"AssetExists","Args":[' + '\"' + id + '\"' + ']}'
    callopt= '{"function":"AssetExists","Args":' + getArgList([id]) + '}'
    return query(peer,callopt)
        
def deleteAsset(peer, asset='asset1'):
    callopt= '{"function":"DeleteAsset","Args":[' + '\"' + asset + '\"' + ']}'
    return invoke(peer,callopt)
    


def readAssetTest(peer, asset='asset1'):
    callopt= '{"function":"ReadAssetTest","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)
    
def updateAsset(peer,arglist=None):
    callopt= '{"function":"UpdateAsset","Args":' + getArgList(asset) + '}'
    return invoke(peer,callopt)


def getSubmittingClientIdentity(peer, user='User1'):
    callopt= '{"function":"GetSubmittingClientIdentity","Args":[' + ']}'
    return query(peer,callopt,user=user)

def getMSPID(peer):
    callopt= '{"function":"GetMSPID","Args":[' + ']}'
    return query(peer,callopt)   


def appendRole(peer,user='',role='Buyer'):
    #print(getArgList([role,name]))
    #callopt= '{"function":"AppendRole","Args":' + getArgList([role,name]) + '}'
    name = getSubmittingClientIdentity(peer, user=user)[:-1]
    callopt= '{"function":"AppendRole","Args":' + getArgList([name,role]) + '}'
    return invoke(peer,callopt)

def getRoles(peer,name=''):
    #callopt= '{"function":"GetRoles","Args":[' + ']}'
    callopt= '{"function":"GetRoles","Args":' + getArgList([name]) + '}'
    return query(peer,callopt)

def getAllRoles(peer):
    callopt= '{"function":"GetAllRoles","Args":[' + ']}'
    return query(peer,callopt)




### Enroll users

In [ ]:
import subprocess
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
   

def uploadFromLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                          '-T', local_path + '/' + local_name, 
                                          'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)   
                    
#downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

def downloadToLocalCmdList(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', remote_name='' ,remote_path='/'):  
    return ['curl', '--user',user + ':' + pwd, 'http://' + host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]


In [ ]:
# download walletcreator to peers
def uploadWalletToolToSharepoint():
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='/')
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

def installPeerWalletTool(peer):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
    #executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
    executePeerCmd(peer.pod_name,'apk add openssl')
    executePeerCmd(peer.pod_name,'apk add openjdk8-jre')
    # chmod 755
    
def createPeerUser(peer,user, admin='admin',
    cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
    key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
        cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
               '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
        return executePeerCmd(peer.pod_name,toCmd(cmd))
    
def installWalletTool():
    _out = widgets.Output()
    with _out:
        for key, peer in peers.items():
            installPeerWalletTool(peer)
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/cert-creator.sh', '-o', '/tmp/cert-creator.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/cert-creator.sh')
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/create-user.sh', '-o', '/tmp/create-user.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/create-user.sh')
    
def addUser(peer,user,org,org2):
    _out = widgets.Output()
    with _out:
        #strlist = ['/tmp/create-user.sh', user, peer.name + '.kubernetes.research.dev.seeburger.de']
        strlist = ['/tmp/create-user.sh', user,org,org2 ]
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peer.pod_name, 'cd /tmp && ' + cmd)    
    return output,_out    

In [308]:
def getPeerUsers(peer,org):
    #strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + peer.name + '.kubernetes.research.dev.seeburger.de/users/']
    strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + org + '/users/']
    #addUser(peers['aubonmoulin'],'Tony.Dubois','aubonmoulin.fr','aubonmoulin.kubernetes.research.dev.seeburger.de')[1] 
    
    print(toCmd(strlist))
    users=[]
    lines=executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 
    for line in lines:
        #users.append(line.split('@')[0])
        users.append(line)
    return users

def getPeerOrganizations(peer):
    strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
    return executePeerCmd(peers[peernames[0]].pod_name,toCmd(strlist)).splitlines() 

def get_peer_of_user(user):
    for key,peer in peers.items():
        orgs=getPeerOrganizations(peer)
        for org in orgs:
            print(peer.pod_name, org)
            users = getPeerUsers(peer,org)
            for ouser in users:
                if user == ouser:
                    return peer

In [310]:
get_peer_of_user('Tony.Dubois@aubonmoulin.fr').__dict__


/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
aubonmoulin-67b4649968-jlskd aubonmoulin.fr
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/


{'name': 'aubonmoulin',
 'pod_name': 'aubonmoulin-67b4649968-jlskd',
 'listen_port': '32432',
 'gossip_port': ''}

In [290]:
orgs=getPeerOrganizations(peers[peernames[0]])

for org in orgs:
    users = getPeerUsers(peers[peernames[0]],org)
    print(org,users)


/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
aubonmoulin.fr ['Tony.Dubois@aubonmoulin.fr']
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.kubernetes.research.dev.seeburger.de/users/
aubonmoulin.kubernetes.research.dev.seeburger.de ['Admin@aubonmoulin.kubernetes.research.dev.seeburger.de', 'Alexandre.Payet@aubonmoulin.kubernetes.research.dev.seeburger.de', 'Stephanie.Hoarau@aubonmoulin.kubernetes.research.dev.seeburge

In [ ]:
installWalletTool()

In [ ]:
addUser(peers['aubonmoulin'],'Tony.Dubois','aubonmoulin.fr','aubonmoulin.kubernetes.research.dev.seeburger.de')
addUser(peers['aubonmoulin'],'Stephanie.Hoarau','hotelsaintdenis.re','aubonmoulin.kubernetes.research.dev.seeburger.de')
addUser(peers['aubonmoulin'],'Alexandre.Payet','majolieboutique.net','aubonmoulin.kubernetes.research.dev.seeburger.de')
addUser(peers['austriafactor'],'Emma.Steiner','austriafactor.at','austriafactor.kubernetes.research.dev.seeburger.de')    
addUser(peers['megustaolive'],'Pedro.Sanchez','megustaolive.es','megustaolive.kubernetes.research.dev.seeburger.de')  
addUser(peers['lyon-impots'],'Auguste.Boucher','lyon-impots.gouv.fr','lyon-impots.kubernetes.research.dev.seeburger.de')  

In [ ]:
/mnt/conf/organizations/peerOrganizations/aubonmoulin.kubernetes.research.dev.seeburger.de/users/User1@aubonmoulin.kubernetes.research.dev.seeburger.de/msp/tlscacerts/tlsca.aubonmoulin.fr-cert.pem'

In [ ]:
peer=peers[peernames[0]]
addUser(peers[peernames[0]],'Tony.Dubois')    
addUser(peers[peernames[0]],'Stephanie.Hoarau')
addUser(peers[peernames[0]],'Alexandre.Payet')
appendRole(peer,user='Tony.Dubois',role='Seller')
appendRole(peer,user='Stephanie.Hoarau',role='Buyer')
appendRole(peer,user='Alexandre.Payet',role='Buyer')

peer=peers[peernames[1]]
addUser(peers[peernames[1]],'Emma.Steiner')    
appendRole(peer,user='Emma.Steiner',role='Factor')

peer=peers[peernames[2]]
addUser(peers[peernames[2]],'Pedro.Sanchez')  
appendRole(peer,user='Pedro.Sanchez',role='Buyer')
appendRole(peer,user='Pedro.Sanchez',role='Seller')

peer=peers[peernames[3]]
addUser(peers[peernames[3]],'Auguste.Boucher')   
appendRole(peer,user='Auguste.Boucher',role='TaxInspector')

In [ ]:
getAllRoles(peer)

In [ ]:
getSubmittingClientIdentity(peers[peernames[0]], user='Tony.Dubois')[0][:-1].split(',')

### Monitoring

In [ ]:
peers['lyon-impots'].__dict__


In [ ]:
getRoleTransactions(peer)[0]

In [342]:
keys = getAllKeys(peer)[0]
keys

['asset14', 'asset17', 'asset3', 'asset6']

In [ ]:
getAllRoles(peer)[0][:-1]
#getRoles(peer,name=name)

In [ ]:
strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
executePeerCmd(peers[peernames[0]].pod_name,toCmd(strlist)).splitlines() 

In [ ]:
getHostname(peers[peernames[0]],'Tony.Dubois@aubonmoulin.fr')

In [ ]:
strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + peer.name + '.kubernetes.research.dev.seeburger.de/users/']
print(toCmd(strlist))
users=[]
lines=executePeerCmd(peers[peernames[0]].pod_name,toCmd(strlist)).splitlines() 

In [ ]:
peers[peernames[0]].pod_name

In [ ]:
#getSubmittingClientIdentity(peers[peernames[1]],user='ben')[0]

get_certificate_string(peernames[1],user='ben')

In [ ]:
name = getSubmittingClientIdentity(peer)[0][:-1].split(',')[0]
group=getSubmittingClientIdentity(peer)[0][:-1].split(',')[1]
msp=getMSPID(peers[peernames[1]])[0][:-1]

name,group,msp

### Block

In [ ]:
import subprocess
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     

In [ ]:
import base64

def get_height():
    global _out
    _out = widgets.Output()
    with _out:
        strlist = ['/mnt/conf/peer/block-operator.sh', 'info', '--channel', channel_name]
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
        return json.loads(output)['height']

def uploadBlockToShare(id):
    strlist = ['/mnt/conf/peer/block-operator.sh fetch', '--channel', channel_name, '--block', str(id),'--publish', shared_fs]
    cmd = toCmd(strlist)
    output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
    
def getBlockFromShare(id):
    filename = 'invoicing-block-' + str(id) + '.block.json'
    downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray', local_path='/tmp/',local_name=filename, remote_path='blocks/invoicing')
    output = subprocess.check_output(['cat', '/tmp/' + filename])
    return json.loads(output.decode('ascii'))           
    
def decode_block_invoice(block):
    _transaction = block['data']['data'][0]['payload']['data']['actions'][0]['payload']['action']['proposal_response_payload']['extension']['results']['ns_rwset'][1]['rwset']['writes'][0]
    if _transaction['value'] != None:
        return json.loads(base64.b64decode(_transaction['value']).decode("utf-8"))
    
def decode_block_transaction(block):
    _transaction = _block['data']['data'][0]['payload']['data']['actions'][0]['payload']['chaincode_proposal_payload']['input']['chaincode_spec']['input']['args']
    result = []
    for arg in _transaction:
        result.append(base64.b64decode(arg).decode("utf-8"))
    return result        

In [398]:
get_height()

17

In [ ]:
uploadBlockToShare(15)    
_block = getBlockFromShare(15)
#_block

In [ ]:
_block['data']['data'][0]['payload']['data']['actions'][0]['payload']['action']['proposal_response_payload']['extension']['results']['ns_rwset'][1]['rwset']
_block['data']['data'][0]['payload']['data']['actions'][0]['payload']['chaincode_proposal_payload']['input']['chaincode_spec']['input']['args']

In [397]:
decode_block_invoice(_block)
decode_block_transaction(_block)

['CreateInvoice',
 'asset17',
 'x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.kubernetes.research.dev.seeburger.de,O=aubonmoulin.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
 'x509::CN=Alexandre.Payet@majolieboutique.net,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.kubernetes.research.dev.seeburger.de,O=aubonmoulin.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
 '6265861722676072020',
 'AV-2017-0005',
 '-14.99',
 '-218.48',
 'FR',
 'FR',
 '',
 'False',
 'False',
 'False',
 'False',
 '',
 '',
 'False']

### create invoice

In [275]:
import unicodedata
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [ ]:
get_certificate_string(peernames[0])

In [ ]:
getPeerUsers(peers[peernames[0]],'aubonmoulin.fr')[0]

In [ ]:
peernames

In [ ]:
peer = peers['aubonmoulin']
getEmptyInvoice(peer)[0]
_user=getPeerUsers(peers[peernames[0]],'aubonmoulin.fr')[0]
#_user='Stephanie.Hoarau@hotelsaintdenis.re'
#_user='User1'
getSubmittingClientIdentity(peers[peernames[0]],user=_user)

In [ ]:
getSubmittingClientIdentity(peer,user=_user)

In [ ]:
peer = peers['aubonmoulin']
asset = getEmptyInvoice(peer)[0]
asset['ID']='asset21'
_user=getPeerUsers(peers[peernames[0]],'aubonmoulin.fr')[0]
#_user='Stephanie.Hoarau@hotelsaintdenis.re'
getSubmittingClientIdentity(peer,user=_user)[0]
#_user

In [ ]:
peer = peers['aubonmoulin']
asset = getEmptyInvoice(peer)[0]
asset['ID']='asset21'
_user=getPeerUsers(peers[peernames[0]],'aubonmoulin.fr')[0]
asset['owner'] = getSubmittingClientIdentity(peer,user=_user)[0]
_user='Stephanie.Hoarau@hotelsaintdenis.re'
asset['buyer'] = getSubmittingClientIdentity(peer,user=_user)[0]

In [ ]:
seller=getPeerUsers(peers[peernames[0]],'aubonmoulin.fr')[0]
createInvoice(peer,getArgList(asset),user=seller)[1]

In [239]:
deleteAsset(peer,asset='asset21')

('', Output())

In [ ]:
getSubmittingClientIdentity(peers[peernames[0]], user='User1')[0]

In [ ]:
keys = getAllKeys(peer)[0]
keys

In [ ]:
keys = getAllKeys(peer)[0]
for key in keys:
    if key.startswith('asset'):
        deleteAsset(peer,asset=key)[1]

In [ ]:
i=16
invoice = invoices.iloc[16]
seller = strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
buyer = strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
print(i,seller,buyer,'asset'+str(i))
update_invoice(id=i,seller=seller,buyer=buyer)
#createInvoice(get_peer_of_user(seller),getArgList(asset),user=seller)[1]
asset

In [ ]:
asset['taxExemptionReason']='French VAT exemption according to articles 262 ter I (for products) and/or 283-2 (for services) of CGI'
createInvoice(get_peer_of_user(seller),getArgList(asset),user=seller)[1]

In [269]:
import pandas as pd

def get_invoices():
    pd.set_option('display.max_rows', 500)
    return pd.read_parquet('./invoices_facturx.parquet')

invoices = get_invoices()

In [278]:
def getPathValue(m,path=''):
    if(len(path)<2):
        return m
    splits = path[1:].split("/", 1)
    value=m.get(splits[0])
    #print(splits,value)
    if not isinstance(value, type(None)):
        if len(splits) > 1:
            return getPathValue(value,path='/' + splits[1])
    return value          

In [318]:
def get_CN_of_invoice(invoice,party):
    _name=strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
    _email=getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['emailURIUniversalCommunication']['uriid']['value']
    return _name + '@' + _email.split('@')[1]

seller = get_CN_of_invoice(invoice,'sellerTradeParty')
getSubmittingClientIdentity(get_peer_of_user(seller), user=seller)[0]

#get_CN_of_invoice(invoice,'buyerTradeParty')
#get_peer_of_user(get_CN_of_invoice(invoice,'buyerTradeParty')).__dict__

/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
aubonmoulin-67b4649968-jlskd aubonmoulin.fr
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/


'x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.kubernetes.research.dev.seeburger.de,O=aubonmoulin.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

In [323]:
def assign_invoice_to_asset(_invoice,asset):
    #print(asset)

    asset['hash'] = hash(repr(sorted(_invoice.items()))) 
    monetary = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/specifiedTradeSettlementHeaderMonetarySummation/')
    asset['invoiceNumber'] = getPathValue(_invoice,'/exchangedDocument/id')['value']
    asset['netto'] = monetary['taxBasisTotalAmount'][0]['value']
    asset['tax'] = monetary['taxTotalAmount'][0]['value']
    
    try:
        asset['taxExemptionReason']  = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/applicableTradeTax')[0]['exemptionReason']['value']
    except Exception as e:
        asset['taxExemptionReason']  = ''
 
    asset['countryBuyer'] = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/postalTradeAddress/countryID')['value']
    asset['countryOrigin']   = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/postalTradeAddress/countryID')['value']
    

In [338]:
def update_invoice(invoice=None,id=None,seller='Tony.Dubois',buyer='Stephanie.Hoarau'):
    assign_invoice_to_asset(invoice,asset)
    asset['ID']='asset' + str(id)
    asset['owner'] = getSubmittingClientIdentity(get_peer_of_user(seller), user=seller)[0]
    asset['buyer'] = getSubmittingClientIdentity(get_peer_of_user(buyer), user=buyer)[0]
    asset['taxExemptionReason'] = strip_accents(asset['taxExemptionReason'])

In [339]:
asset = getEmptyInvoice(peer)[0]
invoice = invoices.iloc[14]
seller = get_CN_of_invoice(invoice,'sellerTradeParty')
buyer  = get_CN_of_invoice(invoice,'buyerTradeParty')
#assign_invoice_to_asset(invoice,asset)
update_invoice(invoice=invoice,id=1,seller=seller,buyer=buyer)
asset


/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
aubonmoulin-67b4649968-jlskd aubonmoulin.fr
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
aubonmoulin-67b4649968-jlskd aubonmoulin.fr
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
aubonmoulin-67b4649968-jlskd aubonmoulin.kubernetes.research.dev.seeburger.de
/bin/ls /mnt/conf/organizations/

{'ID': 'asset1',
 'owner': 'x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.kubernetes.research.dev.seeburger.de,O=aubonmoulin.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
 'buyer': 'x509::CN=Stephanie.Hoarau@hotelsaintdenis.re,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.kubernetes.research.dev.seeburger.de,O=aubonmoulin.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
 'hash': -608721609128042938,
 'invoiceNumber': 'FA-2017-0009',
 'tax': '0.00',
 'netto': '530.75',
 'countryOrigin': 'FR',
 'countryBuyer': 'RE',
 'status': '',
 'received': False,
 'receivedOrder': False,
 'sold': False,
 'claimPaid': False,
 'claimPaidBy': '',
 'taxExemptionReason': 'Exoneration de TVA selon article 262 I du Code general des impots',
 'taxReceived': False}

In [ ]:
for i in range(0,20):
    try:
        invoice = invoices.iloc[i]
        #seller = strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
        #buyer = strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
        seller = get_CN_of_invoice(invoice,'sellerTradeParty')
        buyer  = get_CN_of_invoice(invoice,'buyerTradeParty')
        print(i,seller,buyer,'asset'+str(i))
        asset = getEmptyInvoice(peer)[0]
        update_invoice(invoice=invoice,id=i,seller=seller,buyer=buyer)
        print(asset)
        createInvoice(get_peer_of_user(seller),getArgList(asset),user=seller)[0]
    except Exception as e:
        pass

In [317]:
invoice = invoices.iloc[14]
seller = strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))

In [370]:
def get_cert_cn(cert):
    return cert.split('=')[1].split(',')[0]


In [375]:
asset
#peer = get_peer_of_user(asset['buyer'])
#receivedInvoice(peer, id='asset1', user='User1'):
#peer._dict__
user=get_cert_cn(asset['buyer'])
peer=get_peer_of_user(user)
receivedInvoice(peer, id=asset['ID'], user=user)

/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
aubonmoulin-67b4649968-jlskd aubonmoulin.fr
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
aubonmoulin-67b4649968-jlskd aubonmoulin.kubernetes.research.dev.seeburger.de
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.kubernetes.research.dev.seeburger.de/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-67b4649968-jlskd -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.kubernetes.research.dev.seeburger.de/users/
aubonmoulin-67b4649968-jlskd hotelsaintdenis.re
/bin/ls /mnt/conf/organizations/peerOrganizations/hotelsaintdenis.re/users/
/home/jovyan/work/usr/bin/kubectl exec

('', Output())

In [288]:
peers

{'aubonmoulin': <__main__.kubernetespeer at 0x7fb634344fd0>,
 'lyon-impots': <__main__.kubernetespeer at 0x7fb6343443d0>,
 'austriafactor': <__main__.kubernetespeer at 0x7fb634344700>,
 'megustaolive': <__main__.kubernetespeer at 0x7fb63443f6d0>}

In [ ]:

    
createInvoice(peer,getArgList(asset),user='Tony.Dubois')[1]

In [ ]:
getPathValue(invoices.iloc[14],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/definedTradeContact/')[0]['personName']['value']
getPathValue(invoices.iloc[14],'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/definedTradeContact/')[0]['personName']['value']

### list

In [ ]:
#peer=peers[peernames[1]]
#getSubmittingClientIdentity(peer)[0]
listInvoices(peer=peers[peernames[0]], user='Tony.Dubois')[0]
#listInvoices(peer=peers[peernames[0]])[0]
#listInvoices(peer)[0]
#keys

In [ ]:
#listInvoice(peer=peers[peernames[1]],  user='ben',id=keys[0])[0]

#assetExists(peer=peers[peernames[1]],id=keys[0])

#listInvoice(peer=peers[peernames[0]],  id=keys[0])[0]
readAssetTest(peer=peers[peernames[0]], asset='asset16')

In [ ]:
asset=readAsset(peer, asset='asset1')
asset['sold'] = True
updateAsset(peer, asset)

### received

In [ ]:
receivedInvoice(peer=peers[peernames[1]],  user='ben',id=keys[0])[1]

In [ ]:
peers[peernames[1]].__dict__

### load data

In [ ]:
import json

def load(filename):
    with open(filename) as json_file:
        return json.load(json_file)

invoices = load('invoices.json')
roles = load('roles.json')
roleTransactions = load('roleTransactions.json')

In [ ]:
def get_cert_cn(cert):
    return cert.split('=')[1].split('@')[0]

get_cert_cn(roles[0]['name']), roles[0]['roles']

In [ ]:
def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(get_cert_cn(person['name']))
    return result        
        
get_person_names_with_role('TaxInspector',roles)        

In [ ]:
roles

In [ ]:
if 'TaxInspector' in roles[0]['roles']:
    print()

In [ ]:
def invoke(peer,callopt,user='User1'):
    print(user,callopt)
    
def query(peer,callopt,user='User1'):
    print(user,callopt)    
    
class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name    
        
def addPeer(peername):  
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer        
  
def get_peer_of_user(name):
    return kubernetespeer(name=name)    

def addUser(peer,user):
    if peer.name in peerusers:
        peerusers[peer.name].append(user)
    else:
        peerusers[peer.name] = [user]

peers={}
peerusers ={}

addPeer('aubonmoulin')
addPeer('lyon-impots')
addPeer('austriafactor')
addPeer('megustaolive') 
addUser(peers['aubonmoulin'],'Tony.Dubois')    
addUser(peers['aubonmoulin'],'Stephanie.Hoarau')
addUser(peers['aubonmoulin'],'Alexandre.Payet')
addUser(peers['austriafactor'],'Emma.Steiner')    
addUser(peers['megustaolive'],'Pedro.Sanchez')  
addUser(peers['lyon-impots'],'Auguste.Boucher')  

peernames = list(peers.keys())

In [ ]:
peerusers
invoices[2]

## display

In [ ]:
#asset=listInvoices(peer=peers[peernames[2]], user='Tony.Dubois')[0][1]['Record']
asset=invoices[3]
_widget = facturx_widgets(asset,'/', hideUnused=True) 
display(_widget)

In [ ]:
#listInvoices(peer=peers[peernames[0]], user='Tony.Dubois')[0][0]['Record']

In [ ]:
#_widget.container.fields['ID'].widget.disabled=True



In [ ]:
owner=_widget.container.fields['owner'].widget.value.split(',')[0].split('=')[1].split('@')[0]
buyer=_widget.container.fields['buyer'].widget.value.split(',')[0].split('=')[1].split('@')[0]
taxinspector=get_person_names_with_role('TaxInspector',roles)
id=_widget.container.fields['ID'].widget.value
peer_buyer=get_peer_of_user(buyer)
peer_owner=get_peer_of_user(owner)
peer_taxinspector=get_peer_of_user(taxinspector)

In [ ]:
def disable_all_fields():
    for key,value in _widget.container.fields.items():
        value.widget.disabled=True
        
def enable_fields_for_role(rolename):
    if rolename == 'Buyer':
        _widget.container.fields['received'].widget.disabled=False
        _widget.container.fields['receivedOrder'].widget.disabled=False

    elif  rolename == 'Seller':   
        _widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        _widget.container.fields['owner'].widget.disabled=False
        
    elif  rolename == 'Factor':   
        #_widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        #_widget.container.fields['owner'].widget.disabled=False
                
    elif  rolename == 'TaxInspector':    
        _widget.container.fields['taxReceived'].widget.disabled=False       

In [ ]:
disable_all_fields()
enable_fields_for_role('Buyer')

In [ ]:
disable_all_fields()
enable_fields_for_role('Seller')

In [ ]:
# sold to factor
_widget.container.fields['owner'].widget.value = roles[2]['name']
_widget.container.fields['sold'].widget.value = True

In [ ]:
disable_all_fields()
enable_fields_for_role('Factor')

In [ ]:
_widget.container.fields['claimPaidBy'].widget.value = 'Christophe Monet'
_widget.container.fields['claimPaid'].widget.value = True

In [ ]:
disable_all_fields()
enable_fields_for_role('TaxInspector')

### role buyer

In [ ]:


peer=get_peer_of_user(buyer)

def received_changed(b):
    if b['name'] == 'value':
        receivedInvoice(peer_buyer, id=id, user=buyer)[1]
    
def receivedOrder_changed(b):
    if b['name'] == 'value':
        print('changed')
        receivedOrder(peer_buyer, id=id, user=buyer)[1]    
    
_widget.container.fields['received'].widget.observe(received_changed)
_widget.container.fields['receivedOrder'].widget.observe(receivedOrder_changed)

### role owner / seller

In [ ]:

def sold_changed(b):
    if b['name'] == 'value':
        print('changed')
        #so(peer, id=id, user=seller)[1]
    
def claimPaid_changed(b):
    if b['name'] == 'value':
        print('changed')
        #receivedOrder(peer, id=id, user=seller)[1]    
    
_widget.container.fields['sold'].widget.observe(received_changed)
_widget.container.fields['claimPaid'].widget.observe(receivedOrder_changed)    

### role tax inspector

In [ ]:


def taxReceived_changed(b):
    if b['name'] == 'value':
        taxReceived(peer_taxinspector, id=id, user=taxinspector)[1]
    
_widget.container.fields['taxReceived'].widget.observe(taxReceived_changed)

In [ ]:
receivedInvoice(peers[peernames[0]], id='asset14', user='Stephanie.Hoarau')[1]

### save assets as file

In [ ]:
#asset = readAsset(peers[peernames[0]], asset='asset6')

from facturxWidget import facturx_widgets 

import json
with open('asset.json', 'w', encoding='utf-8') as f:
    json.dump(asset, f, ensure_ascii=False, indent=4)

In [ ]:
import json

with open('asset.json') as json_file:
    data = json.load(json_file)

data    

In [ ]:
assets=listInvoices(peer=peers[peernames[0]], user='Tony.Dubois')[0]

In [ ]:
roleTransactions = getRoleTransactions(peer)[0]
roleTransactions

In [ ]:
roles=getAllRoles(peer)[0]

In [ ]:
import json
with open('roles.json', 'w', encoding='utf-8') as f:
    json.dump(roles, f, ensure_ascii=False, indent=4)

### work

In [ ]:
#peers
peer = peers['aabby-0']
#getHostname(peer)

In [ ]:
#owner = readAssetTest(peer,'asset6') ['owner']
name = getSubmittingClientIdentity(peer)[:-1]
#appendRole(peer,name=name,role='Seller')
appendRole(peer,name=name,role='TaxInspector')

In [ ]:
asset=listInvoices(peer)[0][0]['Record']

In [ ]:
#deleteAsset(peer,asset="roles")[1]
#assetExists(peer,asset="roles")
#readAsset(peer,asset="roles")[1]
#getAllRoles(peer)
#getRoles(peer,name=name)
#getSubmittingClientIdentity(peer)

In [ ]:
#peer.pod_name
deleteAsset(peer, asset=keys[3])

## invoice usecase (Simple)

In [ ]:
def getPeer0Address(peer):
    addr='peer0.kubernetes.research.dev.seeburger.de'
    return str(addr + ':' + peer.listen_port)

def readAsset(peer, asset='asset1'):
    #callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    callopt= '{"function":"ReadAsset","Args":[' + '\"' + asset + '\"' + ']}'
    cmd0 = str('\'' + callopt + "\'")
    try:
        #cmd1= 'CORE_PEER_ADDRESS=\"peer0.kubernetes.research.dev.seeburger.de:31866\" && ' 
        #cmd1 = 'CORE_PEER_ADDRESS=\"' + getPeer0Address(peer) + '\" && '
        cmd1 = 'CORE_PEER_MSPCONFIGPATH=' + path + '\" && '
        #cmd2 = ['peer', 'chaincode','query', '-C', channel_name, '-n', 'basic', '-c']
        cmd2 = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem', '-C',channel_name, '-n', 'basic', '-c']
   
        cmd = toCmd(cmd2) + ' ' + cmd0
        print(cmd)
        #output = executePeerCmd(peer.pod_name, cmd1 + cmd)
        #print(output)
        if output[0] == '{':
            return json.loads(output)
        return cmd
    except Exception as e:
        print(e)
        
asset = readAsset(peers[pernames[0]], 'asset1')
executePeerCmd(peers[pernames[0]].pod_name,asset)

In [ ]:

pernames = list(peers.keys())
chaincode_add_invoice(peers[pernames[0]],channel_name,'046',pernames[1])

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t galya-0-6cc848b49-pv2dn -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh super13 006 x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US

In [ ]:
get_certificate_string(pernames[1])

#'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

#, 'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

In [ ]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  $CHANNEL_NAME $INVOICE_ID
def chaincode_read_invoice(peer,channel_name):
    strlist = ['/mnt/conf/peer/get-my-invoices.sh', channel_name]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 
chaincode_read_invoice(peers[pernames[0]],channel_name)

In [ ]:
def chaincode_transfer_invoice(peer,channel_name,invoice_id,new_owner):
    strlist = ['/mnt/conf/peer/transfer_invoice.sh', channel_name,invoice_id,get_certificate_string(new_owner)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

pernames = list(peers.keys())
chaincode_transfer_invoice(peers[pernames[0]],channel_name,'045',pernames[2])

In [ ]:
get_certificate_string(pernames[2])
NEW_OWNER='x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/transfer_invoice.sh  super43 045 $NEW_OWNER

In [ ]:
kubectl exec --stdin --tty 'lalya-0-7c887cffb8-fpw7h' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-1-7c76dff8c-pzx78' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-2-5f8b45dbc4-52tt7' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045


In [ ]:
peers[pernames[2]].pod_name

## add channel

In [ ]:
orderer.pod_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-d8b6656f7-q8g95 -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh invoicing orderer.example.com 30081

In [ ]:
add_channel('super20')

In [ ]:
name = 'super30'
cmd = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050'])
cmd, orderer.pod_name

In [ ]:
executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')

In [ ]:
executePeerCmd(orderer.pod_name, 'peer channel list', cli='scray-orderer-cli')

In [ ]:
cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh super20 orderer.example.com 7050']
executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

In [ ]:
#cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c','/mnt/conf/orderer/scripts/create_channel.sh super7 orderer.example.com 7050']

#cmd2 = toCmd(['ls','-a'])
cmd2 = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', 'super7', 'orderer.example.com', '7050'])

cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c',cmd2]


print(toCmd(cmd1))
#subprocess.check_output(cmd1)
subprocess.run(cmd1, stdout = subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -it orderer-org1-scray-org-5f97c57d44-l6d5l -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super6 orderer.example.com 7050


In [ ]:
orderer.pod_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'orderer-org1-scray-org-5f97c57d44-j5mbs' -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh 'super' orderer.example.com '7050'

In [ ]:
# Create Peer

peername='dalya8'
addPeer(peername)
peers[peername].__dict__

In [ ]:
##


In [ ]:
executeKubectlCmd(['get', 'pods','-l', 'app=orderer-org1-scray-org','-o','jsonpath=\'{.items[*].status.podIP}\''])  

In [ ]:
peers[peername].__dict__

In [ ]:
orderer.__dict__

## Export channel configuration

In [ ]:
#https://github.com/scray/scray/blob/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/README.md#export-data

SHARED_FS_HOST='10.14.128.38:30080'
_result = SHARED_FS_HOST + '/channel/configuration/$CHANNEL_NAME/config.json'

#kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/publish_channel_conf.sh $CHANNEL_NAME $SHARED_FS_HOST

def export_channel_configuration(peer,channel_name,SHARED_FS_HOST):
    strlist = ['/mnt/conf/orderer/scripts/publish_channel_conf.sh', channel_name,SHARED_FS_HOST]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

#_peer = peers[list(peers.keys())[0]]
_peer = orderer
channel_name = 'super'
export_channel_configuration(_peer,channel_name,SHARED_FS_HOST)

In [ ]:
#list(peers)[0]


### display certificate 

In [ ]:
# Cerificates
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.x509.oid import NameOID

#https://stackoverflow.com/questions/16899247/how-can-i-decode-a-ssl-certificate-using-python

def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    cmd = ['curl', '--user',user + ':' + pwd, 
           'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt']
    res = executeLocalCmd(cmd)
    if 'CERTIFICATE' in str(res):
        return res
       
def getOrgCrt(org=''):    
    pem_data=getCrt(org)
    if pem_data:
        return x509.load_pem_x509_certificate(pem_data, default_backend())

    
def getcrt(peer,user='User1'):
    strlist = ['cat',  ' /tmp/crt_target/hans/user.crt']
    
    #           '/mnt/conf/organizations/peerOrganizations/aabby-0.kubernetes.research.dev.seeburger.de/users/hans@aabby-0.kubernetes.research.dev.seeburger.de/msp/signcerts/hans@aabby-0.kubernetes.research.dev.seeburger.de-cert.pem']
    cmd = toCmd(strlist)
    #print(cmd)  
    output = executePeerCmd(peer.pod_name,  cmd)
    return output,_out    

In [ ]:
crt=getcrt(peer)[0][:-1]
str(crt)

In [ ]:
x509.load_pem_x509_certificate(crt, default_backend())